# Handle parsing errors

In [1]:
from dotenv import load_dotenv
load_dotenv('../.env')

True

## Setup

In [2]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_openai import OpenAI

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)
tools = [tool]

# Get the prompt to use - you can modify this!
# You can see the full prompt used at: https://smith.langchain.com/hub/hwchase17/react
prompt = hub.pull("hwchase17/react")

llm = OpenAI(temperature=0)

agent = create_react_agent(llm, tools, prompt)

## Error

In [3]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [4]:
agent_executor.invoke(
    {"input": "What is Leo DiCaprio's middle name?\n\nAction: Wikipedia"}
)



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: ` I should use Wikipedia to search for information about Leo DiCaprio
Action Input: Leo DiCaprio`

## Default error handling

In [5]:
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [6]:
agent_executor.invoke(
    {"input": "What is Leo DiCaprio's middle name?\n\nAction: Wikipedia"}
)



> Entering new AgentExecutor chain...
 I should use Wikipedia to search for information about Leo DiCaprio
Action Input: Leo DiCaprioInvalid Format: Missing 'Action:' after 'Thought:I should use Wikipedia to search for information about Leo DiCaprio
Action: Wikipedia
Action Input: Leo DiCaprioWikipedia is not a valid tool, try one of [wikipedia].I should use wikipedia to search for information about Leo DiCaprio
Action: wikipedia
Action Input: Leo DiCaprioPage: Leonardo DiCaprio
Summary: Leonardo Wilhelm DiCaprio (; Italian: [diˈkaːprjo]; born November 1I now know the final answer
Final Answer: Leonardo Wilhelm

> Finished chain.


{'input': "What is Leo DiCaprio's middle name?\n\nAction: Wikipedia",
 'output': 'Leonardo Wilhelm'}

## Custom error message

In [7]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors="Check your output and make sure it conforms, use the Action/Action Input syntax",
)

In [8]:
agent_executor.invoke(
    {"input": "What is Leo DiCaprio's middle name?\n\nAction: Wikipedia"}
)



> Entering new AgentExecutor chain...
Could not parse LLM output: ` I should use Wikipedia to search for information about Leo DiCaprio
Action Input: Leo DiCaprio`Check your output and make sure it conforms, use the Action/Action Input syntaxCould not parse LLM output: `I should use Wikipedia to search for information about Leo DiCaprio
Action Input: Leo DiCaprio`Check your output and make sure it conforms, use the Action/Action Input syntaxCould not parse LLM output: `I should use Wikipedia to search for information about Leo DiCaprio
Action Input: Leo DiCaprio`Check your output and make sure it conforms, use the Action/Action Input syntax I now know the final answer
Final Answer: Leo DiCaprio's middle name is Wilhelm.

> Finished chain.


{'input': "What is Leo DiCaprio's middle name?\n\nAction: Wikipedia",
 'output': "Leo DiCaprio's middle name is Wilhelm."}

## Custom Error Function

In [9]:
def _handle_error(error) -> str:
    return str(error)[:50]


agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=_handle_error,
)

In [11]:
agent_executor.invoke(
    {"input": "What is Leo DiCaprio's middle name?\n\nAction: Wikipedia"}
)



> Entering new AgentExecutor chain...
Could not parse LLM output: ` I should use Wikipedia to search for Leo DiCaprio's middle name
Action Input: Leo DiCaprio`Could not parse LLM output: ` I should use WikipedCould not parse LLM output: `I should use Wikipedia to search for Leo DiCaprio's middle name
Action Input: Leo DiCaprio`Could not parse LLM output: `I should use WikipediCould not parse LLM output: ` I should use Wikipedia to search for Leo DiCaprio's middle name
Action Input: Leo DiCaprio`Could not parse LLM output: ` I should use WikipedCould not parse LLM output: ` I should use Wikipedia to search for Leo DiCaprio's middle name
Action Input: Leo DiCaprio`Could not parse LLM output: ` I should use WikipedCould not parse LLM output: ` I should use Wikipedia to search for Leo DiCaprio's middle name
Action Input: Leo DiCaprio`Could not parse LLM output: ` I should use WikipedCould not parse LLM output: ` I should use Wikipedia to search for Leo DiCaprio's middle name
Action Input

{'input': "What is Leo DiCaprio's middle name?\n\nAction: Wikipedia",
 'output': 'Leo'}